### Data Cleaning Log
1. **Dropped Rows:** Removed 1 row with `NaN` in the `description` column. 
   * **Reasoning:** Without a description, we cannot extract sponsor data, making the row useless for this specific analysis.
2. **Date Conversion:** Converted `published_at` to datetime objects.
   * **Reasoning:** Required to perform time-series analysis (e.g., "Views over Time").

In [19]:
import pandas as pd
import os
import re

# --- CONFIGURATION ---
# Define paths
raw_path = os.path.join("..", "Data", "raw", "h3_podcast_raw.csv")
processed_dir = os.path.join("..", "Data", "processed")
processed_path = os.path.join(processed_dir, "h3_podcast_processed.csv")

# Create output directory if it doesn't exist
os.makedirs(processed_dir, exist_ok=True)

# --- 1. LOAD & CLEAN ---
print("Loading raw data...")
df = pd.read_csv(raw_path)

# Drop rows with no description (cannot extract sponsors)
df_clean = df.dropna(subset=['description']).copy()
# Convert dates
df_clean['published_at'] = pd.to_datetime(df_clean['published_at'])

print(f"Cleaned Data Shape: {df_clean.shape}")

# --- 2. SPONSOR EXTRACTION LOGIC ---
# Dictionary of Regex patterns for known sponsors
sponsor_patterns = {
    "GamerSupps": r"gamersupps",
    "NordVPN": r"nordvpn",
    "ExpressVPN": r"expressvpn",
    "Raycon": r"raycon",
    "Honey": r"joinhoney",
    "Native": r"nativedeo",
    "Stamps.com": r"stamps\.com",
    "MeUndies": r"meundies",
    "Manscaped": r"manscaped",
    "Teddy Fresh": r"teddyfresh\.com",
    "Liquid IV": r"liquidiv",
    "HelloFresh": r"hellofresh",
    "Mint Mobile": r"mintmobile",
    "DraftKings": r"draftkings",
    "Shopify": r"shopify",
    "Factor": r"go\.factor"
}

def find_sponsors(text):
    if not isinstance(text, str): 
        return "None"
    
    found = []
    text_lower = text.lower()
    
    for brand, pattern in sponsor_patterns.items():
        if re.search(pattern, text_lower):
            found.append(brand)
            
    # Return as a comma-separated string (easier for CSVs)
    return ", ".join(found) if found else "None"

print("Extracting Sponsors...")
df_clean['sponsors'] = df_clean['description'].apply(find_sponsors)
df_clean['has_sponsor'] = df_clean['sponsors'] != "None"

# --- 3. CATEGORIZATION LOGIC (REFINED) ---
def categorize_episode(title):
    t = title.lower()
    
    # Helper for whole word matching
    def contains_word(keywords, text):
        pattern = r"\b(" + "|".join(map(re.escape, keywords)) + r")\b"
        return bool(re.search(pattern, text))

    # TIER 3: SAFE / COMEDY
    safe_keywords = [
        "green screen", "silent library", "you laugh", "laugh you", 
        "guess who", "who is high", "makeover", "queen of melrose", 
        "live show", "greek theatre", "holiday", "thanksgiving", 
        "christmas", "halloween", "bachelor", "gatsby", 
        "soundbite bracket", "tournament"
    ]
    if contains_word(safe_keywords, t):
        return "Tier 3: Safe/Comedy"

    # TIER 1: HIGH CONFLICT
    risk_keywords = [
        "lawsuit", "kavanaugh", "frogan", "denims", "allegations", "sued", 
        "suing", "defamation", "mrbeast", "colleen", "ballinger", "grooming", 
        "assault", "sa", "investigation", "exposed", "crime", "prison", 
        "scam", "fraud", "hasan", "piker", "antisemitism", "war", "genocide",
        "debate", "conflict", "trigger", "shapiro", "steven crowder", 
        "keemstar", "fresh and fit", "divorce", "victim", "predator",
        "boogie2988", "fousey", "arrested"
    ]
    if contains_word(risk_keywords, t):
        return "Tier 1: High Conflict"

    # TIER 4: GUEST
    guest_keywords = [
        "interview", "featuring", "howie mandel", "jeff wittek", 
        "steiny", "marc rebillet", "alexa nikolas", "beavo", 
        "oliver tree", "pokimane", "stavros", "whitney cummings"
    ]
    if "ft." in t or "feat" in t or contains_word(guest_keywords, t): 
        return "Tier 4: Guest"

    # TIER 2: STANDARD
    return "Tier 2: Standard H3"

print("Categorizing Episodes...")
df_clean['category'] = df_clean['title'].apply(categorize_episode)

# --- 4. CALCULATE METRICS ---
# Like Ratio (Proxy for Sentiment)
df_clean['like_ratio'] = df_clean['like_count'] / df_clean['view_count']
# Engagement

Loading raw data...
Cleaned Data Shape: (199, 7)
Extracting Sponsors...
Categorizing Episodes...
